# Import CSV

In [1]:
import pandas as pd

In [2]:
fileName = 'pilots.csv'
filePath = '../input/'

data = pd.read_csv(filePath + fileName) 

# Remove entries without name
data = data.dropna(subset = ['Full name'])
data = data.reset_index(drop = True)

# Import XML

In [3]:
from xml.dom import minidom

In [4]:
fileName = 'registrations.fsdb'
filePath = '../input/'

xmldata = minidom.parse(filePath + fileName)

In [5]:
pilots = xmldata.getElementsByTagName('FsParticipant')
names = []
sponsors = []
for elem in pilots:
    names.append(elem.attributes['name'].value)
    sponsors.append(elem.attributes['sponsor'].value)

pilots = xmldata.getElementsByTagName('FsCustomAttribute')
teams = []
for elem in pilots:
    if elem.attributes['name'].value == 'klub':
        teams.append(elem.attributes['value'].value)
    
# remove missing names
sponsors = [s for s, n in zip(sponsors, names) if n != '']
teams = [t for t, n in zip(teams, names) if n != '']
names = [n for n in names if n != '']

# Merge XML to pandas dataframe with CSV

In [6]:
# check order (could be that CSV and XML pilot ordering is different)
cols = [[],[]]
for index, pilot in data.iterrows():
    pilotName = pilot['Full name']
    for n, s, t in zip(names, sponsors, teams):
        if pilotName == n :
            cols[0].append(s)
            cols[1].append(t)

In [7]:
sponsors = pd.DataFrame({'Sponsor': cols[0]})
teams = pd.DataFrame({'Teams': cols[1]})

data = pd.concat([data, sponsors], axis = 1)
data = pd.concat([data, teams], axis = 1)

# NOT SURE WHY, but row 0 is added again
# Remove entries without name
data = data.dropna(subset = ['Full name'])

# Create PDFs

In [8]:
from fpdf import FPDF
import datetime

In [9]:
fontSize = 14
fontHeight = 7
titleSize = 20
footerSize = 8
bigSpace = 15
midSpace = 7
smallSpace = 5
emptyPages = 20

In [10]:
title = 'METULJMANIA OPEN 2019'
subtitle = 'Application form'
emptyLine = '____________'
meetDirector = 'DJP Metulj Rimske Toplice, Dušan Durkovič'
responsibility = 'Participation in this competition is at the sole responsibility of each individual pilot. The event organizer does not claim any responsibility for the pilots or other third parties.'

In [11]:
class PDF(FPDF):
    def header(self):
        # Logo
        self.image(filePath + 'logo.png', 10, 8, 33)
        
        # TITLE
        pdf.add_font('Bold', '', '../input/fonts/font_bold.ttf', uni=True)
        pdf.set_font('Bold', '', 8)
        self.set_font('Bold', '', titleSize)
        # Calculate width of title and position
        w = self.get_string_width(title) + 6
        w *= 0.8
        self.set_x((210 - w) / 2)
        self.cell(w, 9, title,'C')
        
        # Line break
        self.ln(10)
        
        # SUBTITLE
        self.set_font('Reg', '', titleSize)
        w = self.get_string_width(subtitle) + 6
        w *= 0.8
        self.set_x((210 - w) / 2)
        self.cell(0, 9,subtitle,'C')
        
        # Line break
        self.ln(30)

    def footer(self):
        # Position at 1.5 cm from bottom
        self.set_y(-15)
        pdf.add_font('Reg', '', '../input/fonts/font_reg.ttf', uni=True)
        pdf.set_font('Reg', '', 8)
        self.set_font('Reg', '', footerSize)
        # Text color in gray
        self.set_text_color(128)
        # Page number
        
        self.cell(0, 10, 'Document is automatically generated with FPDF. Author: Mitja Jančič', 0, 0, 'C')

In [12]:
def getValueFromData(PDF, Pilot, Property, FriendlyName, FontSize, LineBreak, EmptyLine = '', FontHeight = 7):
    PDF.set_font('Reg', '', FontSize)
    PDF.write(FontHeight, FriendlyName)
    PDF.set_font('Bold', '', FontSize)
    
    if Property == 'NULL':
        entry = EmptyLine
    else:
        entry = str(Pilot[Property])
        
    if (entry == 'nan'):
        entry = EmptyLine
        
    PDF.write(FontHeight, entry)
    PDF.ln(LineBreak)

In [13]:
pdf = PDF()
pdf.set_left_margin(20)
pdf.set_right_margin(20)
pdf.add_font('Reg', '', '../input/fonts/font_reg.ttf', uni=True)
pdf.add_font('Bold', '', '../input/fonts/font_bold.ttf', uni=True)

for index, pilot in data.iterrows():
    pdf.add_page()
    
    # Pilot name
    getValueFromData(pdf, pilot, 'Full name', 'Pilot name: ', fontSize, midSpace)
    # Gender
    getValueFromData(pdf, pilot, 'Sex', 'Gender: ', fontSize, midSpace)
    # Licence no.
    getValueFromData(pdf, pilot, 'Licence no.', 'Licence no.: ', fontSize, midSpace)
    # FAI Sporting license
    getValueFromData(pdf, pilot, 'FAI number', 'FAI number: ', fontSize, midSpace)
    # CIVL ID
    getValueFromData(pdf, pilot, 'CIVL', 'CIVL ID: ', fontSize, midSpace)
    # Date of birth
    getValueFromData(pdf, pilot, 'Date of birth', 'Date of birth: ', fontSize, midSpace)
    # Mobile phone
    getValueFromData(pdf, pilot, 'Mobile phone', 'Mobile phone: ', fontSize, midSpace)    
    # flying since
    getValueFromData(pdf, pilot, 'NULL', 'Flying since: ', fontSize, midSpace, EmptyLine = emptyLine)
    # team
    getValueFromData(pdf, pilot, 'Teams', 'Team: ', fontSize, midSpace, EmptyLine = 2 * emptyLine)
    # sponsor
    getValueFromData(pdf, pilot, 'Sponsor', 'Sponsors: ', fontSize, midSpace)
    # nationality
    getValueFromData(pdf, pilot, 'Country', 'Nation: ', fontSize, midSpace)
    
    # glider
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontHeight, 'Paraglider (EN 926/2): ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontHeight, str(pilot['Manufacturer']) + ' ' + str(pilot['Glider']))
    pdf.ln(midSpace)
    
    # glider class
    getValueFromData(pdf, pilot, 'Class', 'Safety class: ', fontSize, midSpace)
    # glider color
    getValueFromData(pdf, pilot, 'NULL', 'Paraglider color: ', fontSize, bigSpace, EmptyLine = emptyLine)
    
    # health insurance
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Medical insurance, which includes paragliding competitions:')
    pdf.ln(10)
    getValueFromData(pdf, pilot, 'NULL', 'Company: ', fontSize, midSpace, EmptyLine = 2*emptyLine)
    getValueFromData(pdf, pilot, 'NULL', 'Card/policy number: ', fontSize, bigSpace, EmptyLine = 2*emptyLine)
    
    # Responsibility
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Responsibility:')
    pdf.ln(10)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(7, responsibility)
    pdf.ln(bigSpace)
    
    # Organisor
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Meet director: ')
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, meetDirector)
    pdf.ln(midSpace)
    
    # Signature
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Signature by organizer: ')
    pdf.ln(bigSpace)
    
    # Date created
    pdf.ln(smallSpace)
    pdf.set_font('Reg', '', 10)
    date = datetime.datetime.now().strftime("%I:%M:%S %p on %B %d, %Y.")
    pdf.write(fontSize, 'Application form auto-generated at ' + date)


In [14]:
for i in range(emptyPages):
    pdf.add_page()
    
    # Pilot name
    getValueFromData(pdf, pilot, 'NULL', 'Pilot name: ', fontSize, midSpace,)
    # Gender
    getValueFromData(pdf, pilot, 'NULL', 'Gender: ', fontSize, midSpace)
    # Licence no.
    getValueFromData(pdf, pilot, 'NULL', 'Licence no.: ', fontSize, midSpace)
    # FAI Sporting license
    getValueFromData(pdf, pilot, 'NULL', 'FAI number: ', fontSize, midSpace)
    # CIVL ID
    getValueFromData(pdf, pilot, 'NULL', 'CIVL ID: ', fontSize, midSpace)
    # Date of birth
    getValueFromData(pdf, pilot, 'NULL', 'Date of birth: ', fontSize, midSpace)
    # Mobile phone
    getValueFromData(pdf, pilot, 'NULL', 'Mobile phone: ', fontSize, midSpace)    
    # flying since
    getValueFromData(pdf, pilot, 'NULL', 'Flying since: ', fontSize, midSpace)
    # team
    getValueFromData(pdf, pilot, 'NULL', 'Team: ', fontSize, midSpace)
    # team
    getValueFromData(pdf, pilot, 'NULL', 'Sponsors: ', fontSize, midSpace)
    # nationality
    getValueFromData(pdf, pilot, 'NULL', 'Nation: ', fontSize, midSpace)
    
    # glider
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontHeight, 'Paraglider (EN 926/2): ')
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontHeight, '')
    pdf.ln(midSpace)
    
    # glider class
    getValueFromData(pdf, pilot, 'NULL', 'Safety class: ', fontSize, midSpace)
    # glider color
    getValueFromData(pdf, pilot, 'NULL', 'Paraglider color: ', fontSize, bigSpace)
    
    # health insurance
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Medical insurance, which includes paragliding competitions:')
    pdf.ln(10)
    getValueFromData(pdf, pilot, 'NULL', 'Company: ', fontSize, midSpace)
    getValueFromData(pdf, pilot, 'NULL', 'Card/policy number: ', fontSize, bigSpace)
    
    # Responsibility
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Responsibility:')
    pdf.ln(10)
    pdf.set_font('Reg', '', fontSize)
    pdf.write(7, responsibility)
    pdf.ln(bigSpace)
    
    # Organisor
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Meet director: ')
    pdf.set_font('Reg', '', fontSize)
    pdf.write(fontSize, meetDirector)
    pdf.ln(midSpace)
    
    # Signature
    pdf.set_font('Bold', '', fontSize)
    pdf.write(fontSize, 'Signature by organizer: ')
    pdf.ln(bigSpace)
    
    # Date created
    pdf.ln(smallSpace)
    pdf.set_font('Reg', '', 10)
    date = datetime.datetime.now().strftime("%I:%M:%S %p on %B %d, %Y.")
    pdf.write(fontSize, 'Application form auto-generated at ' + date)

In [15]:
pdf.output('../output/ApplicationForms.pdf', 'F')

''